In [32]:
#initialise

import extractor
extractor.main()



Audio extracted to output_data/audio.mp3
Extracted 16959 frames.
MemoryError detected! Falling back to streaming...
Features saved to output_data/output_features.csv


In [33]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor  

In [34]:
df = pd.read_csv('output_data/output_features.csv')
df['frame_index'] = df.index
df['normalized_index']=df['frame_index']/len(df)
df = df.sample(frac=1.0,random_state=42).reset_index(drop=True)
print(df[['frame_filename','frame_index','normalized_index']].head())

                              frame_filename  frame_index  normalized_index
0   output_frames/frame_4802_time_200283.jpg         4802          0.283153
1   output_frames/frame_4959_time_206831.jpg         4959          0.292411
2  output_frames/frame_12083_time_503961.jpg        12083          0.712483
3  output_frames/frame_12508_time_521687.jpg        12508          0.737543
4   output_frames/frame_3024_time_126126.jpg         3024          0.178312


In [35]:
# Check shape
print("DataFrame shape:", df.shape)

# Check first few rows
print(df.head())

# Check for missing rows or NaNs
print(df.isnull().sum())

# Check if you accidentally only have one row
if df.shape[0] <= 1:
    print("Warning: Only 1 row detected!")


X = df.drop(['timestamp','frame_filename','frame_index','normalized_index',], axis=1)
y = df['normalized_index']

DataFrame shape: (16959, 1516)
   timestamp                             frame_filename  hist_bin_0  \
0     200283   output_frames/frame_4802_time_200283.jpg    0.064454   
1     206831   output_frames/frame_4959_time_206831.jpg    0.162910   
2     503961  output_frames/frame_12083_time_503961.jpg    0.008784   
3     521687  output_frames/frame_12508_time_521687.jpg    0.232377   
4     126126   output_frames/frame_3024_time_126126.jpg    0.001261   

   hist_bin_1  hist_bin_2  hist_bin_3  hist_bin_4  hist_bin_5  hist_bin_6  \
0    0.000008    0.000000    0.000000    0.000000    0.000000    0.000000   
1    0.009885    0.001956    0.000146    0.000477    0.000418    0.000263   
2    0.002387    0.001613    0.001193    0.001150    0.001462    0.000731   
3    0.001040    0.005187    0.018644    0.002651    0.002270    0.000939   
4    0.003557    0.001619    0.002842    0.004122    0.006889    0.002334   

   hist_bin_7  ...  edge_992  edge_993  edge_994  edge_995  edge_996  \
0    0.

In [36]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

model.fit(X_train, y_train)


RandomForestRegressor(n_jobs=-1, random_state=42)

In [37]:
y_pred = model.predict(X_test)

In [38]:
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE: {mse:.4f}")
print(f"R2 Score: {r2:.4f}")

MSE: 0.0017
R2 Score: 0.9802


In [39]:
df['predicted_normalized_index'] = model.predict(X)

# sort by predicted order
df_sorted = df.sort_values(by='predicted_normalized_index')

print(df_sorted[['frame_filename','predicted_normalized_index']].head(10))


                             frame_filename  predicted_normalized_index
1417   output_frames/frame_30_time_1251.jpg                    0.001784
2762   output_frames/frame_31_time_1292.jpg                    0.001844
2703   output_frames/frame_33_time_1376.jpg                    0.001946
6838   output_frames/frame_32_time_1334.jpg                    0.002017
3736   output_frames/frame_35_time_1459.jpg                    0.002195
6349   output_frames/frame_37_time_1543.jpg                    0.002214
4958   output_frames/frame_36_time_1501.jpg                    0.002260
4916   output_frames/frame_39_time_1626.jpg                    0.002288
15171  output_frames/frame_38_time_1584.jpg                    0.002347
2647   output_frames/frame_41_time_1710.jpg                    0.002419


In [40]:
import cv2

fps = 23.98 # adjust as needed
frame_size = (640,360) # adjust as needed
output_video = cv2.VideoWriter('output_data/reconstructed.avi',
                               cv2.VideoWriter_fourcc(*'XVID'),
                               fps,
                               frame_size)


for _, row in df_sorted.iterrows():
    frame = cv2.imread(row['frame_filename'])
    output_video.write(frame)

output_video.release()
print("Reconstructed Video Saved!!")

Reconstructed Video Saved!!


In [41]:
import subprocess

def combine_audio_video(reconstructed_video, audio_file, output_video):
    command = ['ffmpeg', '-i', reconstructed_video, '-i', audio_file, '-c:v', 'copy', '-c:a', 'aac', output_video, '-y']
    subprocess.run(command, check=True)
    print(f"Final video saved to {output_video}")

combine_audio_video('output_data/reconstructed.avi', 'output_data/audio.mp3','output_data/final_output.mp4')

Final video saved to output_data/final_output.mp4
